# RNA-seqの解析練習

## チャ Camellia sinensis のモチ病（Blister Blight）
基本情報
- チャ: ツバキ科ツバキ属の木本植物
- チャもち病菌: Exobasidium vexans; 担子菌類
- チャもち病発生地域: 茶の生産地域（スリランカ、インド、インドネシア、日本など）で発生する
- 被害: 茶の品質低下、収量低下を引き起こす

## RNA-seqサンプル
_Jayaswall, K., Mahajan, P., Singh, G., Parmar, R., Seth, R., Raina, A., ... & Sharma, R. K. (2016). Transcriptome analysis reveals candidate genes involved in blister blight defense in tea (Camellia sinensis (L) Kuntze). Scientific reports, 6, 30412._

- 抵抗性品種 SA6
- 感受性品種 Kangra Asha
- 病原菌接種14日後
のRNA-seqサンプル  
    病原菌が（感受性品種の）植物細胞に侵入して植物から栄養を搾取する吸器を伸ばす時期

## データ概要
### 発現比較データ（./tea_set/S3.dataset.txt）


In [4]:
import pandas as pd
df = pd.read_csv('./tea_set/S3.dataset.txt', sep='\t', header=0, index_col=0)
df.head()

#================================
#    - 遺伝子名
#    - S3_RG: 抵抗性品種の遺伝子発現量（TMM正規化後）
#    - S3_SG: 感受性品種の遺伝子発現量（TMM正規化後）
#    - logFC: log2(抵抗性品種/感受性品種)の値  
#        正の値なら抵抗性品種の発現量が高い  
#        負の値なら感受性品種の発現量が高い  
#    - logCPM: log2((抵抗性品種 + 感受性品種)/2)の値  
#        この値が大きいほど、発現量の大きい遺伝子である指標
#    - PValue: Fisher's exact testによる有意確率（p-value）  
#    - FDR: Benjamini-Hochberg法によるFalse Discovery Rate（FDR）  
#        通常、この値が0.05より小さい場合、有意である（発現量に差がある）とみなす。

,S3_RG,S3_SG,logFC,logCPM,PValue,FDR
TRINITY_DN7059_c0_g2,740.059,0.108,12.590532,8.799814,2.650976e-26,5.956743e-22
TRINITY_DN7059_c0_g1,1268.262,0.637,10.689974,6.902134,1.306793e-20,1.468182e-16
TRINITY_DN6936_c2_g1,232.299,0.143,10.530966,6.743582,3.896130e-20,2.918201e-16
TRINITY_DN8566_c3_g2,112.268,0.000,13.314056,6.349821,5.356733e-20,3.009145e-16
TRINITY_DN7894_c0_g1,79.082,0.000,13.268257,6.304171,7.344590e-20,3.300659e-16


---

### 発現変動遺伝子のBLAST結果リスト（./tea_set/S3.DEGs_BLAST.txt） 

In [5]:
import pandas as pd
df_BLAST = pd.read_csv('./tea_set/S3.DEGs_BLAST.txt', sep='\t', header=0, index_col=0)
df_BLAST.head()

#================================
#    - gene_id: 遺伝子名
#    - BLASTX_swissprot:  
#        UniProtKB/Swiss-Protデータベースに対してBLASTXをして、ベストスコアとして得られた情報

,BLASTX_swissprot
TRINITY_DN7059_c0_g2,"RD22_ARATH^RD22_ARATH^Q:965-288,H:161-390^49.7..."
TRINITY_DN7059_c0_g1,.
TRINITY_DN6936_c2_g1,.
TRINITY_DN8566_c3_g2,.
TRINITY_DN7894_c0_g1,"NATT3_THANI^NATT3_THANI^Q:911-1435,H:181-354^2..."


---
### 発現変動遺伝子の塩基配列（./tea_set/S3.DEGs.fasta）

In [6]:
import pandas as pd
FASTA = open('./tea_set/S3.DEGs.fasta', 'r')

# データフレームに変換
seqs = { 'id':[], 'seq':[] }
sw = 0
for line in FASTA:
    sw += 1
    line = line.rstrip()
    if sw % 2 == 1:
        seqs['id'].append(line[1:])
    else:
        seqs['seq'].append(line)
df_FASTA = pd.DataFrame(seqs).set_index(['id'])
df_FASTA.head()

#================================
#    - id : 遺伝子名
#    - seq: 塩基配列

,seq
id,
TRINITY_DN7059_c0_g2_i1,TTTTTTTAAGCATATAAAGTTGGTTGAAGTTATTAATTAATATAAA...
TRINITY_DN7059_c0_g1_i1,GCATCCTTATCCCCTCTGCCCACGTTGTAAAGAATCGCTCTTGCAT...
TRINITY_DN6936_c2_g1_i1,TCGGCGTTGGGTGTGTTCGACCATAGGAGTGGAGAAGTGGAGTCGG...
TRINITY_DN8566_c3_g2_i1,GCCAAGTGATATGGTGATTCCGGAGTATTTACGCCATTATGTTAGG...
TRINITY_DN7894_c0_g1_i1,AGTGAGAGAGAGAGAGAGAGAGAGAGAGAGAATGGCACTGCCAAGG...


---
## 今回の解析内容
1. 発現量を比較した遺伝子の総数は？

1. 有意な発現（FDR < 0.05）とみなせる遺伝子数は？

1. 発現に差がある遺伝子のうち、S3_RG（抵抗性品種）で高発現の遺伝子数は？

1. 発現に差がある遺伝子のうち、S3_SG（感受性品種）で高発現の遺伝子数は？

1. S3_RG（抵抗性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？

1. S3_SG（感受性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？  
    
1. もち病への抵抗性-感受性を決めている遺伝子（候補）は何か？


---

病原菌抵抗性関連遺伝子のキーワード
- resistance
- pathogen
- LRR (Leucine-rich repeat)
- WRKY
- NAC domain
- chitinase
- peroxidase
- pectinesterase inhibitor
- jasmonate-induced protein

データベース/ツール
- NCBI BLAST 配列相同性検索: https://blast.ncbi.nlm.nih.gov/Blast.cgi
- Uniprot 遺伝子/タンパク質データベース: https://www.uniprot.org/
- Google scholar 論文検索: https://scholar.google.co.jp/

その他解析ツール
- BLAST検索プログラム

```python
import remote_ncbi as rn
rn.ncbi_blast(BLASTの種類, 塩基配列)

# BLASTの種類: 'blastn', 'blastx', など
```


## 解答
---

### 1. 発現量を比較した遺伝子の総数は？

発現比較データ（./tea_set/S3.dataset.txt）に、各遺伝子の発現量と発現比較結果（logFCや確率など）が書かれている。  
1行に1遺伝子の情報が含まれているため、行数をカウントすれば良い。

```python
import pandas as pd
df = pd.read_csv('./tea_set/S3.dataset.txt', sep='\t', header=0, index_col=0)
```

上のコードにより、発現比較データがdfという名前の変数（Pandasデータフレーム）に入る。

_<sub>Googleなどで`pandas 行数カウント` といったキーワードで検索すれば、情報が得られます。</sub>_

In [7]:
### コード例
len(df)   # データフレームの行数カウント

22470

---
### 2. 有意な発現（FDR < 0.05）とみなせる遺伝子数は？

発現比較データ（./tea_set/S3.dataset.txt）の中で、`FDR<0.05`より小さいデータの数をカウントすれば良い。

_<sub>データフレームでの条件検索方法は、Googleなどで `pandas 条件` といったキーワードで検索すれば、情報が得られます。</sub>_

In [8]:
### コード例
sub = df[df['FDR'] < 0.05]  # FDR<0.05の行のみ抽出
len(sub)                    # 行数カウント

1135

---
### 3. 発現に差がある遺伝子のうち、S3_RG（抵抗性品種）で高発現の遺伝子数は？

「S3_RG（抵抗性品種）で高発現」の場合、`sub`に入っているデータ（`FDR<0.05`の条件に合うデータ）のうち、`logFC>0`の条件に合うデータを抽出して、カウントすればよい。

### 4. 発現に差がある遺伝子のうち、S3_SG（感受性品種）で高発現の遺伝子数は？

「S3_SG（感受性品種）で高発現」の場合、`sub`に入っているデータ（`FDR<0.05`の条件に合うデータ）のうち、`logFC<0`の条件に合うデータを抽出して、カウントすればよい。


In [9]:
### コード例
# 両方の結果を表示するために、print関数を使っています。

subR = sub[sub['logFC']>0]  # subデータフレームからlogFC>0の行のみ抽出
subS = sub[sub['logFC']<0]  # subデータフレームからlogFC<0の行のみ抽出

print('RG:', len(subR), '| SG:', len(subS)) # 行数カウント

RG: 670 | SG: 465


---
### 5. S3_RG（抵抗性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？

「病原菌応答関連遺伝子」とは何か？ => 植物の病原菌応答に関して調べてください。  
__ここでは、「病原菌応答関連遺伝子のキーワード」を使ってください。__

__病原菌抵抗性関連遺伝子のキーワード__
- resistance
- pathogen
- LRR (Leucine-rich repeat)
- WRKY
- NAC domain
- chitinase
- peroxidase
- pectinesterase inhibitor
- jasmonate-induced protein

発現変動遺伝子の詳細な情報（BLAST検索で得られる情報）は `./tea_set/S3.DEGs_BLAST.txt` にあります。  
そのファイル内に、`resistance`や`pathogen`、`LRR`といったワードが出現します。

解析手順
1. `./tea_set/S3.DEGs_BLAST.txt`を読み込む（以下、`df_BLAST`）
1. `subR`と`df_BLAST`のデータをつなぐ（以下、`subR_BLAST`）
1. `subR_BLAST`内でワード検索をおこなう
1. 検索ワードが見つかったデータ行のみを表示する

_<sub>データフレームの結合方法は、Googleなどで `pandas 結合` といったキーワードで検索すれば、情報が得られます。</sub>_  
_<sub>データフレームのワード検索方法は、Googleなどで `pandas 文字列　検索` といったキーワードで検索すれば、情報が得られます。</sub>_

In [148]:
### コード例
# 読み込み
df_BLAST = pd.read_csv('./tea_set/S3.DEGs_BLAST.txt', sep='\t', header=0, index_col=0)

# 確認（最初の数行のみ）
df_BLAST.head()

,BLASTX_swissprot
TRINITY_DN7059_c0_g2,"RD22_ARATH^RD22_ARATH^Q:965-288,H:161-390^49.7..."
TRINITY_DN7059_c0_g1,.
TRINITY_DN6936_c2_g1,.
TRINITY_DN8566_c3_g2,.
TRINITY_DN7894_c0_g1,"NATT3_THANI^NATT3_THANI^Q:911-1435,H:181-354^2..."


In [149]:
# データを結合する
subR_BLAST = pd.merge(subR, df_BLAST, left_index=True, right_index=True, how='left')

#-----
# [補足]
# pd.merge(データフレームleft, データフレームright, 結合方法など)
# インデックス列（遺伝子名の列）を使って結合するので、left_index=True, right_index=True
# subRにある遺伝子のみのデータを得たいので, how='left' 
#-----

subR_BLAST.head(10)  #確認

,S3_RG,S3_SG,logFC,logCPM,PValue,FDR,BLASTX_swissprot
TRINITY_DN7059_c0_g2,740.059,0.108,12.590532,8.799814,2.650976e-26,5.956743e-22,"RD22_ARATH^RD22_ARATH^Q:965-288,H:161-390^49.7..."
TRINITY_DN7059_c0_g1,1268.262,0.637,10.689974,6.902134,1.306793e-20,1.468182e-16,.
TRINITY_DN6936_c2_g1,232.299,0.143,10.530966,6.743582,3.896130e-20,2.918201e-16,.
TRINITY_DN8566_c3_g2,112.268,0.000,13.314056,6.349821,5.356733e-20,3.009145e-16,.
TRINITY_DN7894_c0_g1,79.082,0.000,13.268257,6.304171,7.344590e-20,3.300659e-16,"NATT3_THANI^NATT3_THANI^Q:911-1435,H:181-354^2..."
TRINITY_DN9205_c10_g1,125.801,0.000,12.692644,5.730888,3.765700e-18,1.410255e-14,.
TRINITY_DN8745_c2_g1,58.730,0.000,12.603243,5.641939,6.861158e-18,2.202432e-14,"GDL83_ARATH^GDL83_ARATH^Q:1338-250,H:1-370^54...."
TRINITY_DN9754_c2_g1,3695.645,25.041,7.067319,7.932474,4.331880e-16,9.964653e-13,.
TRINITY_DN11021_c0_g1,105.038,0.000,11.837871,4.881808,1.234881e-15,2.312315e-12,.
TRINITY_DN8633_c3_g1,1194.744,0.000,11.723732,4.768714,2.610205e-15,3.665707e-12,"ANRCS_VITVI^ANRCS_VITVI^Q:231-1,H:37-113^85.71..."


In [150]:
KEY_WORD='resistance'

subR_BLAST[ subR_BLAST['BLASTX_swissprot'].str.contains(KEY_WORD) ]

#-----
# [補足]
# subR_BLAST['BLASTX_swissprot'].str.contains(KEY_WORD) について
# BLASTX_swissprotの列に対して、文字列検索をおこないます。

,S3_RG,S3_SG,logFC,logCPM,PValue,FDR,BLASTX_swissprot
TRINITY_DN2541_c1_g1,2.756,0.000,8.639465,1.781773,5.433481e-07,0.000052,"LRK10_WHEAT^LRK10_WHEAT^Q:1753-80,H:29-628^46...."
TRINITY_DN6404_c1_g1,13.215,7.926,3.016430,4.529390,4.373221e-05,0.002078,"DRL4_ARATH^DRL4_ARATH^Q:407-3,H:4-135^42.647%I..."
TRINITY_DN10112_c0_g1,4.391,0.000,7.459637,0.728913,1.647989e-04,0.006011,"RGA3_SOLBU^RGA3_SOLBU^Q:160-468,H:7-105^33.01%..."
TRINITY_DN7242_c2_g1,22.367,5.666,2.334675,5.345789,9.070866e-04,0.022748,"LRK10_WHEAT^LRK10_WHEAT^Q:2041-410,H:71-634^40..."


---
### 6. S3_SG（感受性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？


In [151]:
### コード例
subS_BLAST = pd.merge(subS, df_BLAST, left_index=True, right_index=True, how='left')
subS_BLAST[  subS_BLAST['BLASTX_swissprot'].str.contains(KEY_WORD)  ]

,S3_RG,S3_SG,logFC,logCPM,PValue,FDR,BLASTX_swissprot
TRINITY_DN7024_c1_g1,0.000,3.730,-10.460294,3.521401,1.066364e-11,4.792240e-09,"R13L1_ARATH^R13L1_ARATH^Q:3729-1360,H:36-823^3..."
TRINITY_DN4358_c0_g1,0.051,6.052,-6.639263,4.337275,1.250716e-11,5.404537e-09,"RP8L3_ARATH^RP8L3_ARATH^Q:3622-848,H:2-887^28...."
TRINITY_DN5288_c0_g1,2.088,5.935,-3.971619,3.709387,1.233207e-06,1.045667e-04,"DRL7_ARATH^DRL7_ARATH^Q:648-439,H:805-871^44.2..."
TRINITY_DN4655_c0_g1,0.000,2.385,-7.821804,1.034280,3.568050e-05,1.727890e-03,"DRL21_ARATH^DRL21_ARATH^Q:663-4,H:1092-1315^39..."
TRINITY_DN7281_c0_g1,0.000,2.448,-7.539396,0.788799,1.255611e-04,4.881242e-03,"RGA3_SOLBU^RGA3_SOLBU^Q:2-112,H:954-990^62.162..."
TRINITY_DN9002_c3_g1,1.265,8.419,-3.169967,2.736069,1.427746e-04,5.474653e-03,"DRL28_ARATH^DRL28_ARATH^Q:514-47,H:767-916^31...."
TRINITY_DN4300_c0_g1,0.000,2.905,-7.019223,0.355220,1.006409e-03,2.421724e-02,"TMVRN_NICGU^TMVRN_NICGU^Q:335-730,H:12-142^56...."


### Uniprotの情報を得る

In [4]:
Entry_Name = 'R13L1_ARATH'

def uniprot_search(Entry_Name):
    import pandas as pd
    import os
    import urllib.request, urllib.parse 
    
    
    url = 'https://www.uniprot.org/uniprot/'
    params = {
        'query'  : 'mnemonic:' + Entry_Name,
        'sort'   : 'score',
        'columns': 'id,entry name,protein names,genes,organism',
        'format' : 'tab',
    }

    req = urllib.request.Request('{}?{}'.format(url, urllib.parse.urlencode(params)))

    with urllib.request.urlopen(req) as res:
        body = res.read()
    
        if not body:
            print(body)
        
        else:
            with open('temp.txt', 'w') as f_tmp:
                f_tmp.write(body.decode())
            df = pd.read_csv('temp.txt', sep='\t', header=0, index_col=0)
            os.remove('temp.txt')
            return df

uniprot_search(Entry_Name)

,Entry name,Protein names,Gene names,Organism
Entry,,,,
Q9LRR4,R13L1_ARATH,Putative disease resistance RPP13-like protein 1,RPPL1 At3g14470 MOA2.9,Arabidopsis thaliana (Mouse-ear cress)


---
### 7. もち病への抵抗性-感受性を決めている遺伝子（候補）は何か？
5,6の結果を元に考察してください。

---
### 発現変動遺伝子のGO Termリスト ~ グラフ

発現変動遺伝子のGO Termリスト `./tea_set/S3.DEGs_GO.txt`

In [5]:
# 読み込み
df_GO = pd.read_csv('./tea_set/S3.DEGs_GO.txt', sep='\t', header=-1, index_col=0, names=['GO'])

subR_GO = pd.merge(subR, df_GO, left_index=True, right_index=True, how='inner')
subS_GO = pd.merge(subS, df_GO, left_index=True, right_index=True, how='inner')

#---
# カウント
import count_go as cg

subR_count = cg.count_go(subR_GO)
subS_count = cg.count_go(subS_GO)

count = pd.merge(subR_count, subS_count, left_index=True, right_index=True, how='outer').fillna(0)
count.columns = ['S3_RG', 'S3_SG']

#---
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot') 
%matplotlib inline

g = count.plot.barh(
    y=count.columns,
    alpha=0.6,
    figsize=(9,int(len(count)*0.2)),
)
g.tick_params(labelsize=8)

NameError: name 'pd' is not defined

In [123]:
GO_ID='GO:0009699'

def go_id2term(GO_ID):
    import requests, sys
    import urllib.parse
    import json

    requestURL = "https://www.ebi.ac.uk/QuickGO/services/ontology/go/search?query=" + urllib.parse.quote(GO_ID) + "&limit=1&page=1"

    r = requests.get(requestURL, headers={ "Accept" : "application/json"})

    if not r.ok:
        r.raise_for_status()
        sys.exit()

    responseBody = r.text

    responseBody = r.text
    d = json.loads(responseBody)

    for res in d["results"]:
        print(res['id'], res['name'], res['aspect'])
    
go_id2term(GO_ID)

count.loc[GO_ID]

GO:0009699 phenylpropanoid biosynthetic process biological_process


S3_RG    6.0
S3_SG    0.0
Name: GO:0009699, dtype: float64

In [127]:
# 抵抗性品種（S3_RG）に特異的なGO Term

countR = count[count['S3_SG']==0].sort_values(by=['S3_RG'], ascending=False)

# 抵抗性品種（S3_RG）に特異的なGO Term

countS = count[count['S3_RG']==0].sort_values(by=['S3_SG'], ascending=False)

print('RG:', len(countR), '| SG:', len(countS))

#countR.head()

RG: 810 | SG: 824


In [101]:
for i in countS.index:
    go_id2term(i)

GO:0032196 transposition biological_process
GO:0048471 perinuclear region of cytoplasm cellular_component
GO:0044463 cell projection part cellular_component
GO:0030162 regulation of proteolysis biological_process
GO:0016701 oxidoreductase activity, acting on single donors with incorporation of molecular oxygen molecular_function
GO:0040011 locomotion biological_process
GO:1901401 regulation of tetrapyrrole metabolic process biological_process
GO:1901463 regulation of tetrapyrrole biosynthetic process biological_process
GO:0090056 regulation of chlorophyll metabolic process biological_process
GO:0031331 positive regulation of cellular catabolic process biological_process
GO:0031329 regulation of cellular catabolic process biological_process
GO:0012506 vesicle membrane cellular_component
GO:0010438 cellular response to sulfur starvation biological_process
GO:0016413 O-acetyltransferase activity molecular_function
GO:0006914 autophagy biological_process
GO:0010380 regulation of chlorophyl

GO:0048284 organelle fusion biological_process
GO:0048280 vesicle fusion with Golgi apparatus biological_process
GO:0048278 vesicle docking biological_process
GO:0048219 inter-Golgi cisterna vesicle-mediated transport biological_process
GO:0019344 cysteine biosynthetic process biological_process
GO:0048211 Golgi vesicle docking biological_process
GO:0019419 sulfate reduction biological_process
GO:0022406 membrane docking biological_process
GO:0022611 dormancy process biological_process
GO:0022627 cytosolic small ribosomal subunit cellular_component
GO:0047162 17-O-deacetylvindoline O-acetyltransferase activity molecular_function
GO:0046903 secretion biological_process
GO:0030659 cytoplasmic vesicle membrane cellular_component
GO:0046459 short-chain fatty acid metabolic process biological_process
GO:0046283 anthocyanin-containing compound metabolic process biological_process
GO:0045862 positive regulation of proteolysis biological_process


KeyboardInterrupt: 